In [11]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import re

cell_regex = r'\",|, "|},' #\d,
dict_regex = r'\":'

In [19]:
def get_normalized_df(df, col, keys, key_attribute="movie_id"):
    df_copy = df.copy()
    df_copy[col] = df_copy[col].map(clean_cell)
    
    clean_dfs = []
    for i, row in df_copy.iterrows():
        movie_id = int(row[key_attribute])
        df_new = pd.DataFrame()
        for key in keys:
            df_new[key] = [sub_col[key] for sub_col in row[col]]
        df_new["id"] = [movie_id]*len(df_new)
        clean_dfs.append(df_new)
    df_clean = pd.concat(clean_dfs, ignore_index=True)

    return df_clean

trim_string = lambda string: "".join([c for c in string if c.isdigit() or c.isalpha() or c==" "]).strip()

def clean_cell(string):
    if len(string)==2: return []
    n_keys = int(string.count(":")/string.count("}"))
    d, l = defaultdict(lambda: None), []
    rows = re.split(cell_regex, string)
    for i, cell in enumerate([row for row in rows if len(row)!=2]):
        k, v = re.split(dict_regex, cell)
        k, v = trim_string(k), trim_string(v)
        d[k] = v
            
        if i % n_keys == 0 and i != 0:
            l.append(d)
            d = defaultdict(lambda: None)
    return l

In [20]:
df_movies = pd.read_csv("data/movies_groupA.csv")
df_credits = pd.read_csv("data/credits_groupA.csv")
df_recom = pd.read_csv("C:/Users/ibele/OneDrive/DHBW/4. Semester/Big Data Storage/Prüfungsleistung/groupA/recommendations_groupA.csv")

In [14]:
df_recom.to_csv("filtered_data/recom.csv")
df_movies[["title","release_date","runtime","revenue","budget","popularity","id"]].to_csv("filtered_data/movies.csv")

In [21]:
new_tables = {
    "spoken_languages": (df_movies, "id", ["name"]),
    "genres": (df_movies, "id", ["name"]),
    "cast": (df_credits, "movie_id", ["name", "character", "gender"]),
    "crew": (df_credits, "movie_id", ["name", "job", "gender"])
}
for table, (df, id_name, cols) in new_tables.items():
    normalized_df = get_normalized_df(df, table, cols, id_name)
    normalized_df.to_csv(f"filtered_data/{table}.csv", index=False)